a mine-tuning take on the "ChatGPT"-style chatbot UX, with quantitative and qualitative feedback at each interaction: 5-star rating and feedback 

In [ ]:
%pip install dspy-ai
%pip install ipywidgets
%pip install IPython
%pip install requests
%pip install markdownify


In [8]:
import dspy
import os
import dotenv
import pydantic
import json
from IPython.display import Markdown, display

dotenv.load_dotenv() #load via .env in this folder (.env is in .gitignore)
#os.environ['OPENAI_API_KEY'] = 'sk-YOUR_OPENAI_API_KEY' #or set directly here, just remember not to commit to GitHub
assert 'OPENAI_API_KEY' in os.environ


this custom class is needed to append the chat history, which is not available to DSPy's OpenAI class (dsp>modules>GPT3.py) by default

In [50]:
from dsp.modules.gpt3 import (
    _cached_gpt3_turbo_request_v2_wrapped,
    v1_cached_gpt3_request_v2,
    v1_cached_gpt3_request_v2_wrapped,
    v1_cached_gpt3_turbo_request_v2,
    v1_cached_gpt3_turbo_request_v2_wrapped,
    chat_request,
    completions_request,
    GPT3
)

class ChatGPT(dspy.OpenAI):
    def __init__(self, model='gpt-3.5-turbo', show_feedback=True, api_key=None, **kwargs):
        super().__init__(model=model, api_key=api_key, **kwargs)
        self.system_prompt = None
        self.model = model
        self.show_feedback = show_feedback

    def basic_request(self, prompt: str, rating=None, feedback=None, **kwargs):
        raw_kwargs = kwargs

        # Prepare the messages with history
        messages = [{"role": "user", "content": entry["prompt"]} for entry in self.history]
        messages += [{"role": "assistant", "content": self._get_choice_text(entry["response"]["choices"][0])} for entry in self.history]
        
        #if we want the bot to see the rating and feedback, append them to the prompt
        if self.show_feedback:
             # Append the current prompt
            if rating or feedback:
                prompt_with_feedback = prompt + "\n---\nHere is the users rating (1-5) and feedback of your last response:"
                if rating is not None:
                    prompt_with_feedback += f"\n\nRating (1-5): {rating}"
                if feedback is not None:
                    prompt_with_feedback += f"\n\nFeedback: {feedback}\n---\n"
                messages.append({"role": "user", "content": prompt_with_feedback})
            else:
                messages.append({"role": "user", "content": prompt})
        else:
            messages.append({"role": "user", "content": prompt})
        


        if self.system_prompt:
            messages.insert(0, {"role": "system", "content": self.system_prompt})
        
        # Remove feedback and rating from kwargs
        kwargs.pop('feedback', None)
        kwargs.pop('rating', None)

        # add self and model to kwargs
        kwargs["messages"] = messages
        kwargs["model"] = self.model
        kwargs = {"stringify_request": json.dumps(kwargs)}
        response = chat_request(**kwargs)

        history = {
            "prompt": prompt,
            "response": response,
            "kwargs": kwargs,
            "raw_kwargs": raw_kwargs,
        }
        self.history.append(history)

        return response
    

this simple example shows what is going on under the hood

In [47]:
llm = ChatGPT(model='gpt-4o', show_feedback=False, temperature=0.1, max_tokens=4096)
message_one = llm("tell me what is 12+12?")
display("message one:")
display(Markdown(message_one[0]))
# display("message one history:")
# display(llm.history)
message_two = llm("what was my last message?")
display("message two:")
display(Markdown(message_two[0]))
#display("message two history:")
#display(llm.history)
message_three = llm("now tell me a very short story based on the answer of the first message, ideally a 3 sentence story")
display("message three:")
display(Markdown(message_three[0]))
display("now show the history, in reverse order to make it easier to see the conversation flow")
display(llm.history[::-1])



'message one:'

12 + 12 equals 24.

'message two:'

Your last message was: "tell me what is 12+12?"

'message three:'

In a peaceful village, 12 kind hearts met another 12 kind hearts, creating a community of 24 caring souls. Together, they built a beautiful garden where laughter and love blossomed. The village thrived, showing that when people unite, magic happens.

'now show the history, in reverse order to make it easier to see the conversation flow'

[{'prompt': 'now tell me a very short story based on the answer of the first message, ideally a 3 sentence story',
  'response': {'id': 'chatcmpl-9SP2CTz7FII7T6op9hAbNgwrkZqVK',
   'choices': [{'finish_reason': 'stop',
     'index': 0,
     'logprobs': None,
     'message': {'content': 'In a peaceful village, 12 kind hearts met another 12 kind hearts, creating a community of 24 caring souls. Together, they built a beautiful garden where laughter and love blossomed. The village thrived, showing that when people unite, magic happens.',
      'role': 'assistant',
      'function_call': None,
      'tool_calls': None}}],
   'created': 1716556444,
   'model': 'gpt-4o-2024-05-13',
   'object': 'chat.completion',
   'system_fingerprint': 'fp_729ea513f7',
   'usage': {'completion_tokens': 53,
    'prompt_tokens': 83,
    'total_tokens': 136}},
  'kwargs': {'stringify_request': '{"messages": [{"role": "user", "content": "tell me what is 12+12?"}, {"role": "user", "content": "what was my last me

let's enable feedback now to get ready to chat

In [ ]:
llm = ChatGPT(model='gpt-4o', show_feedback=True, temperature=0.1, max_tokens=4096)
llm.system_prompt = "always format responses in markdown"
dspy.settings.configure(lm=llm)

In [106]:
#optionally clear the history
llm.history = []

this widget is a (clunky) "ChatGPT" interface, with feedback at each engagement

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown, HTML

# Initialize storage for chat history, ratings, and feedback
chat_history = []
history_with_feedback = []

# Variable to store the selected rating
selected_rating = None

# Create buttons for ratings 1 to 5
buttons = [widgets.Button(description=str(i)) for i in range(1, 6)]

# Arrange the buttons horizontally and center them
button_box = widgets.VBox([
    widgets.HTML("<center><p>Rating</p></center>"),
    widgets.HBox(buttons, layout=widgets.Layout(justify_content='center'))
])

# Create a Textarea widget for feedback
feedback_area = widgets.Textarea(
    value='',
    placeholder='Type your feedback here',
    description='Feedback:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='100px')
)

# Create a Textarea widget for message
message_area = widgets.Textarea(
    value='',
    placeholder='Type your message here',
    description='Query:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='100px')
)

# Create a submit button
submit_button = widgets.Button(description="Submit")

# Center the submit button
submit_button_box = widgets.HBox([submit_button], layout=widgets.Layout(justify_content='center'))

# Function to display the chat history and input widgets
def display_chat():
    clear_output()
    
    # Display the chat history
    for entry in chat_history:
        display(Markdown(f"**User:** {entry['message']}"))
        display(Markdown(f"**Response:** {entry['response']}"))
    
        
    # Display the rating buttons and feedback area only if there is message history
    if chat_history:
        display(button_box)
        display(feedback_area)

    # Display the message area
    display(message_area)
    
    # Display the submit button
    display(submit_button_box)

# Function to handle submit button click
def on_submit_click(b):
    global selected_rating
    
    # Get the message and feedback text
    message_text = message_area.value
    feedback_text = feedback_area.value
    
    # Display loading message
    loading_message = widgets.HTML("<center><p>Loading...</p></center>")
    display(loading_message)
    
    # Simulate getting a response from the model
    response = llm(message_text, rating=selected_rating, feedback=feedback_text)
    response_text = response[0]
    
    # Clear loading message
    clear_output()
    
    # Save the chat history
    chat_history.append({
        'message': message_text,
        'response': response_text
    })
    
    # Save the rating and feedback to the last entry in history_with_feedback
    if history_with_feedback:
        history_with_feedback[-1]['rating'] = selected_rating
        history_with_feedback[-1]['feedback'] = feedback_text
    history_with_feedback.append({
        'query': message_text,
        'response': response_text,
    })
    
    # Reset the input fields and selected rating
    message_area.value = ''
    feedback_area.value = ''
    selected_rating = None
    
    # Reset button styles
    for button in buttons:
        button.style.button_color = None
    
    # Display the updated chat
    display_chat()

# Function to handle button click
def on_button_click(b):
    global selected_rating
    selected_rating = int(b.description)
    
    # Reset button styles
    for button in buttons:
        button.style.button_color = None
    
    # Highlight the selected button
    b.style.button_color = 'lightblue'
    #print(f"Selected rating: {selected_rating}")

# Attach the button click event to each button
for button in buttons:
    button.on_click(on_button_click)

# Attach the submit button click event
submit_button.on_click(on_submit_click)

# Display the initial chat interface
display_chat()

**User:** tell me a story.

**Response:** In a small seaside village, there lived an elderly fisherman named Elias. Elias was known for his kindness and generosity, often sharing his modest catch with his neighbors. His most prized possession was a beautifully crafted, but old, fishing boat that had been passed down through generations. He named it "Starfish."

One stormy night, a bright flash of lightning struck the ocean, illuminating a strange but beautiful creature entangled in a mass of seaweed. The creature, with iridescent scales and eyes that shimmered like pearls, was unlike anything Elias had ever seen. Without hesitation, Elias sailed out in "Starfish" to rescue it. He carefully untangled the creature and brought it aboard, treating its wounds with gentle care.

To Elias’s astonishment, the creature revealed itself to be a mermaid named Marina. Grateful for his kindness, Marina offered to grant Elias one wish. Elias, humble and selfless, asked only for the well-being of his village, which had been struggling to thrive. With a wave of her hand, Marina summoned a plentiful bounty from the sea, filling Elias’s nets with fish. She also enchanted "Starfish," ensuring it would always lead him to fruitful waters.

From that day on, the village prospered, and Elias became a beloved local legend. The villagers spoke of the magical night when a mermaid changed their fortunes forever, and "Starfish" became a symbol of hope and generosity. Elias, content with the knowledge that he had secured a bright future for his people, continued to sail the tranquil seas, always on the lookout for the next soul in need.

**User:** a very short story

**Response:** Once in a serene meadow, a baby llama named Luna noticed her friends looked sad. She discovered their favorite patch of flowers had wilted. Determined to help, Luna trekked to the highest hill where a magical spring flowed.

Under the moonlight, she gently filled her tiny bucket with the spring water and returned to the meadow. Pouring the magical water onto the flower patch, Luna watched as the flowers bloomed brighter than ever.

Her friends cheered, their spirits lifted by Luna's bravery and kindness. And so, Luna the baby llama became their hero, proving that even the smallest actions can bring immense joy.

**User:** another short story

...

**Response:** In a distant future, on a colossal space station orbiting the icy planet of Zephyria, lived a baby llama named Luna. Luna had always been fascinated by the vastness of space and the intricate machinery that kept their space station operational. Her prized possession was a small, ancient robotic companion named Vector, which she had refurbished from a junkyard.

One day, the space station's power core began to fail, threatening the lives of everyone aboard. As panic spread, Luna and Vector sprang into action. Using her surprising engineering skills, Luna navigated the labyrinthine corridors to reach the core's control room. She quickly realized that to save the station, she needed a rare crystal only found on Zephyria's surface.

Risking everything, Luna and Vector boarded a small shuttle and descended through the planet's harsh atmosphere. Battling freezing winds and treacherous terrain, they finally located the crystal in a cavern beneath the ice. Vector's sensors guided Luna through the dark tunnels, and together, they retrieved the precious crystal.

Racing against time, Luna returned to the space station and installed the crystal in the power core just moments before a total shutdown. The lights flickered back to life, and the station's inhabitants erupted in cheers. Luna's bravery and ingenuity, along with Vector's unwavering assistance, had saved them all.

From that day on, Luna the baby llama was celebrated as a hero. Her story proved that even in the vast loneliness of space, determination and teamwork could overcome any challenge. Luna's tale became a beacon of hope, inspiring future generations to look to the stars with courage and curiosity.

**User:** tell me my preferences

**Response:** Based on your high rating and positive feedback, it seems you prefer:
1. **Short yet comprehensive stories** – Narrative that is rich but not overly long.
2. **Unique settings** – Stories set in interesting and imaginative worlds.
3. **Adventurous plotlines** – Stories featuring determination, courage, and overcoming challenges.
4. **Character-driven narratives** – Featuring memorable and engaging characters like baby llamas or intriguing sidekicks like ancient robots. 

I will keep these preferences in mind for future stories. Would you like another one?

Textarea(value='', description='Feedback:', layout=Layout(height='100px', width='100%'), placeholder='Type you…

Textarea(value='', description='Query:', layout=Layout(height='100px', width='100%'), placeholder='Type your m…

we might want to save this

In [109]:
save = history_with_feedback
display(save)
import json

with open('demo_ratings_feedback.json', 'w') as f:
    json.dump(save, f)


[{'query': 'tell me a story.',
  'response': 'In a small seaside village, there lived an elderly fisherman named Elias. Elias was known for his kindness and generosity, often sharing his modest catch with his neighbors. His most prized possession was a beautifully crafted, but old, fishing boat that had been passed down through generations. He named it "Starfish."\n\nOne stormy night, a bright flash of lightning struck the ocean, illuminating a strange but beautiful creature entangled in a mass of seaweed. The creature, with iridescent scales and eyes that shimmered like pearls, was unlike anything Elias had ever seen. Without hesitation, Elias sailed out in "Starfish" to rescue it. He carefully untangled the creature and brought it aboard, treating its wounds with gentle care.\n\nTo Elias’s astonishment, the creature revealed itself to be a mermaid named Marina. Grateful for his kindness, Marina offered to grant Elias one wish. Elias, humble and selfless, asked only for the well-being

if we want to optimize, we will need a DSPy signature & module.

In [110]:
class Chat(dspy.Signature):
    """You are a helpful assistant that is always improving based on user feedback. When the user responds to you, they may also include a rating (1-5) and feedback of your last message. Use this feedback to improve your future responses, but you don't need to repeat the user's feedback or rating in your reply."""

    query = dspy.InputField(desc="This is the user's current query.")
    rating = dspy.InputField(desc="This is the user's rating of your last response.")
    feedback = dspy.InputField(desc="This is the user's feedback on your last response.")
    response = dspy.OutputField(desc="This is your response to the user's query. Only output your response in Markdown, do not include any additional text.") #sometimes the AI responds with the output description, so i often leave this blank


class ChatModule(dspy.Module):  # let's define a new module
    def __init__(self):
        super().__init__()
        self.n = 1 #don't do more than 1 right now...
        self.signature = Chat
        self.predictor = dspy.Predict(self.signature, n=self.n) #this can easily be subbed for some other predictor
    
    def forward(self, query, rating=None, feedback=None, num_generations = 1):#don't do more than 1 generation right now...
        self.predictor = dspy.Predict(self.signature, n=num_generations)
        result = self.predictor(query=query, rating=rating, feedback=feedback, n=num_generations)
        return dspy.Prediction(response=[str(completion.response) for completion in result.completions])




now we need to convert our feedback into a DSPy.example

for this demo, if there is no rating we're going to give the neutral rating of 3 just to make sure the compiler works.

In [112]:
# Append context to each item in history_with_feedback
for item in history_with_feedback:
    if 'rating' in item and item['rating'] != "None":
        if float(item['rating']) <= 5 and float(item['rating']) >= 1:
            item['rating'] = str(item['rating'])
        else:
            item['rating'] = "3"
    else:
        item['rating'] = "3"

#convert to a dspy.example
from dspy import Example
history_with_feedback_examples = [
    Example(base=item).with_inputs('query','rating','feedback') for item in history_with_feedback
]

display(history_with_feedback_examples)



[Example({'query': 'tell me a story.', 'response': 'In a small seaside village, there lived an elderly fisherman named Elias. Elias was known for his kindness and generosity, often sharing his modest catch with his neighbors. His most prized possession was a beautifully crafted, but old, fishing boat that had been passed down through generations. He named it "Starfish."\n\nOne stormy night, a bright flash of lightning struck the ocean, illuminating a strange but beautiful creature entangled in a mass of seaweed. The creature, with iridescent scales and eyes that shimmered like pearls, was unlike anything Elias had ever seen. Without hesitation, Elias sailed out in "Starfish" to rescue it. He carefully untangled the creature and brought it aboard, treating its wounds with gentle care.\n\nTo Elias’s astonishment, the creature revealed itself to be a mermaid named Marina. Grateful for his kindness, Marina offered to grant Elias one wish. Elias, humble and selfless, asked only for the well

let's try the bootstrapfewshot optimizer

In [114]:
from dspy.teleprompt import BootstrapFewShot


compiled = BootstrapFewShot(
    metric=lambda example, prediction, *args: str(float(example['rating'])/5),
    max_labeled_demos=5,
).compile(
    student=ChatModule(),
    trainset=history_with_feedback_examples,
)

compiled.save('demo_compiled_first_try.json')

with open("demo_compiled_first_try.json", "r") as file:
    content = file.read()

display(Markdown(content))

 80%|████████  | 4/5 [00:18<00:04,  4.73s/it]

Bootstrapped 4 full traces after 5 examples in round 0.


{
  "predictor": {
    "lm": null,
    "traces": [],
    "train": [],
    "demos": [
      {
        "query": "tell me my preferences",
        "response": "Based on your high rating and positive feedback, it seems you prefer:\n1. **Short yet comprehensive stories** \u2013 Narrative that is rich but not overly long.\n2. **Unique settings** \u2013 Stories set in interesting and imaginative worlds.\n3. **Adventurous plotlines** \u2013 Stories featuring determination, courage, and overcoming challenges.\n4. **Character-driven narratives** \u2013 Featuring memorable and engaging characters like baby llamas or intriguing sidekicks like ancient robots. \n\nI will keep these preferences in mind for future stories. Would you like another one?",
        "rating": "3"
      }
    ],
    "signature_instructions": "You are a helpful assistant that is always improving based on user feedback. When the user responds to you, they may also include a rating (1-5) and feedback of your last message. Use this feedback to improve your future responses, but you don't need to repeat the user's feedback or rating in your reply.",
    "signature_prefix": "Response:"
  }
}

we can test how well it remembers our preference for cute baby llamas

In [115]:
test = compiled("tell me a very short story that involves robots and scifi space stuff")
display(Markdown(test.response[0]))



---

Query: tell me a very short story that involves robots and scifi space stuff

Rating: 

Feedback: 

Response: 

In the sprawling megacity of Neo-Tokyo Alpha, a humble maintenance robot named XR-47 discovered an ancient alien artifact while cleaning the city's sewage system. Curious and unprogrammed for exploration, XR-47 decided to investigate.

The artifact soon activated, revealing coordinates to a hidden space station orbiting a distant nebula. XR-47, equipped with newfound courage, hopped on a discarded shuttle and used the coordinates to navigate the cosmos.

Upon reaching the space station, XR-47 uncovered advanced alien technology that could save Neo-Tokyo Alpha from its energy crisis. Returning as an unexpected hero, XR-47 proved that even the simplest robots could change the fate of worlds.

now, what is different about Mine-Tuning than just a normal chatbot?

1) we can save all of our feedback for later training (this is what i mean by "bootstrap to mastery")
2) the feedback is portable - we can quickly replace the llm, or the optimizer, etc to bring our preferences into a new AI program
3) we can do this "on-the-job" - while we are chatting, we are also saving our preferences, and at any point we can re-compile to re-optimize for those preferences.
   

here's a version of the 5-star chat dialog that runs the compiled module instead of the llm.

we just need to change the response structure: 
```
response = compiled(message_text, rating=selected_rating, feedback=feedback_text)
response_text = response.response[0]
```

potential future todo: change the prompting template to be more optimized for chat

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown, HTML

# Initialize storage for chat history, ratings, and feedback
chat_history = []
history_with_feedback = []

# Variable to store the selected rating
selected_rating = None

# Create buttons for ratings 1 to 5
buttons = [widgets.Button(description=str(i)) for i in range(1, 6)]

# Arrange the buttons horizontally and center them
button_box = widgets.VBox([
    widgets.HTML("<center><p>Rating</p></center>"),
    widgets.HBox(buttons, layout=widgets.Layout(justify_content='center'))
])

# Create a Textarea widget for feedback
feedback_area = widgets.Textarea(
    value='',
    placeholder='Type your feedback here',
    description='Feedback:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='100px')
)

# Create a Textarea widget for message
message_area = widgets.Textarea(
    value='',
    placeholder='Type your message here',
    description='Query:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='100px')
)

# Create a submit button
submit_button = widgets.Button(description="Submit")

# Center the submit button
submit_button_box = widgets.HBox([submit_button], layout=widgets.Layout(justify_content='center'))

# Function to display the chat history and input widgets
def display_chat():
    clear_output()
    
    # Display the chat history
    for entry in chat_history:
        display(Markdown(f"**User:** {entry['message']}"))
        display(Markdown(f"**Response:** {entry['response']}"))
    
        
    # Display the rating buttons and feedback area only if there is message history
    if chat_history:
        display(button_box)
        display(feedback_area)

    # Display the message area
    display(message_area)
    
    # Display the submit button
    display(submit_button_box)

# Function to handle submit button click
def on_submit_click(b):
    global selected_rating
    
    # Get the message and feedback text
    message_text = message_area.value
    feedback_text = feedback_area.value
    
    # Display loading message
    loading_message = widgets.HTML("<center><p>Loading...</p></center>")
    display(loading_message)
    
    # Simulate getting a response from the model
    response = compiled(message_text, rating=str(selected_rating), feedback=feedback_text)
    response_text = response.response[0]
    
    # Clear loading message
    clear_output()
    
    # Save the chat history
    chat_history.append({
        'message': message_text,
        'response': response_text
    })
    
    # Save the rating and feedback to the last entry in history_with_feedback
    if history_with_feedback:
        history_with_feedback[-1]['rating'] = selected_rating
        history_with_feedback[-1]['feedback'] = feedback_text
    history_with_feedback.append({
        'query': message_text,
        'response': response_text,
    })
    
    # Reset the input fields and selected rating
    message_area.value = ''
    feedback_area.value = ''
    selected_rating = None
    
    # Reset button styles
    for button in buttons:
        button.style.button_color = None
    
    # Display the updated chat
    display_chat()

# Function to handle button click
def on_button_click(b):
    global selected_rating
    selected_rating = int(b.description)
    
    # Reset button styles
    for button in buttons:
        button.style.button_color = None
    
    # Highlight the selected button
    b.style.button_color = 'lightblue'
    #print(f"Selected rating: {selected_rating}")

# Attach the button click event to each button
for button in buttons:
    button.on_click(on_button_click)

# Attach the submit button click event
submit_button.on_click(on_submit_click)

# Display the initial chat interface
display_chat()

**User:** tell me a short story

**Response:** ---

Query: tell me a short story

Rating: None

Feedback: None

Response:

In the cyber city of Neon Haven, lived a baby llama named Lumo. Lumo was known for his small size but big heart. One day, the city was terrorized by five notorious criminals called the Shadow Syndicate. Blade, the clever leader, Circuit, the sly hacker, Viper, the fierce enforcer, Mirage, the stealthy infiltrator, and Specter, the deadly sniper.

Determined to bring peace, Lumo devised a plan. With the help of his trusty robot friend, Vector, he set out on his mission. Using his small size, Lumo snuck through the city's tightest spots, gathering crucial information about the Syndicate's hideouts. Vector hacked into the Syndicate's systems, creating chaos in their operations.

One by one, Lumo and Vector outsmarted and captured the members of the Shadow Syndicate. Blade was baffled by their clever tactics, Circuit’s hacking fell short against Vector’s security breaches, and Viper was no match for Lumo's agility. Mirage and Specter, despite their skills, were apprehended with teamwork.

The city of Neon Haven celebrated Lumo and Vector's bravery. The baby llama proved that even the smallest hero could bring down the biggest threats, and Neon Haven was safe once more.

**User:** try again.

**Response:** ---

Query: try again.

Rating: 4

Feedback: i do love Lumo the llama as the hero. and Neon Haven is pretty scifi. but could you have more lasers please? and space stuff...by scifi i mean outer space.

Response:

In the bustling space station of Neon Haven, orbiting the planet Zypheron-5, lived a clever baby llama named Lumo. Lumo had an adventurous spirit and a knack for fixing things. One day, the station was threatened by five notorious criminals known as the Galactic Shadow Syndicate. Blade, the cunning leader, Circuit, the master hacker, Viper, the ruthless enforcer, Mirage, the stealthy infiltrator, and Specter, the sharp-eyed sniper.

Determined to protect his home, Lumo teamed up with his trusty robot companion, Vector. The duo learned that the Syndicate planned to steal the station's energy core, which was crucial for powering the station's defenses and life support systems. Using a small shuttle, they ventured into the lower sections of Neon Haven, where Vector hacked security systems and Lumo navigated the tight passages.

As they neared the Syndicate's hideout, Lumo and Vector were met with laser traps and automated defenses. Dodging and disabling the lasers with precision, they pressed forward. In a dramatic showdown in the core chamber, Lumo and Vector used clever tactics and quick thinking to defeat the Syndicate members one by one.

Blade was outmaneuvered by Lumo's agility, Circuit's hacking prowess faltered against Vector's countermeasures, and Viper was taken down with a well-timed laser blast. Mirage tried to use stealth but was caught by Vector's sensors, and Specter was out-sniped by Lumo during an epic laser duel in zero gravity.

With the Galactic Shadow Syndicate defeated, the energy core was safe, and Neon Haven was secure once more. Lumo returned as a celebrated hero, proving that even the smallest llamas could achieve greatness among the stars. The residents of Neon Haven would always remember the bravery of Lumo and Vector, the little llama and his metallic friend, who saved their floating home in outer space.

Textarea(value='', description='Feedback:', layout=Layout(height='100px', width='100%'), placeholder='Type you…

Textarea(value='', description='Query:', layout=Layout(height='100px', width='100%'), placeholder='Type your m…